In [1]:
import sys
workdir = '/home/jovyan/workspace'
if not workdir in sys.path:
    sys.path.append(workdir)

In [2]:
import planetengine
system = planetengine.systems.arrhenius.build(res = 16)
initials = {'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}
planetengine.initials.apply(
    initials,
    system,
    )
system.solve()

In [3]:
myvar = planetengine.functions.convert(system.temperatureField)

In [4]:
myvars = {'temperature': myvar}

In [9]:
import numpy as np
myvar = planetengine.functions.convert(system.temperatureField)
myvars = {'temperature': myvar}
with planetengine.tests.TestDir() as outputPath:
    data = myvar.var.data
    planetengine.disk.varsOnDisk(myvars, outputPath, mode = 'save')
    myvars['temperature'].var.data[:] = 0
    planetengine.disk.varsOnDisk(myvars, outputPath, mode = 'load')
    assert np.allclose(data, myvar.var.data[:])
    print("Success!")

Saving substrate to disk: mesh
Saving var to disk: temperature
Loading substrate from disk: mesh
Loading var from disk: temperature
Success!


In [ ]:
# class CustomFrame(_Frame):
#     def __init__(self):

#         system = planetengine.systems.arrhenius.build(res = 16)
#         initials = {'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}
#         planetengine.initials.apply(
#             initials,
#             system,
#             )
#         system.solve()

#         self.system = system
#         self.initials = initials
#         self.outputPath = '/home/jovyan/workspace/data/test'
#         self.instanceID = 'testFrame'
#         self.stamps = {'a': 1}
#         self.step = 0
#         self.onDisk = False
#         self._is_child = False
#         self.inFrames = []
#         self._autoarchive = True
#         checkpointer = checkpoint.Checkpointer(
#             stamps = self.stamps,
#             step = system.step,
#             modeltime = system.modeltime,
#             )
#         mypath = os.path.join(self.outputPath, self.instanceID)
#         def checkpoint(path = None):
#             if path is None:
#                 path = mypath
#             checkpointer.checkpoint(path)
#             self.onDisk = True
#         self.checkpoint = checkpoint
#         def load_checkpoint(step):
#             pass
#         self.load_checkpoint = load_checkpoint

#         super().__init__()

In [ ]:
from planetengine.utilities import check_reqs
from planetengine.utilities import hashstamp
from planetengine.wordhash import wordhash as wordhashFn
from planetengine.frame import _Frame
from planetengine.frame import _scripts_and_stamps

import underworld as uw

class Observer(_Frame):

    _required_attributes = {
        'system', # system
        'initials', # dict
        'outputPath', # str
        'scripts', # list of str
        'inputs', # list of dict
        'figs', # list of glucifer figs
        'collectors', # list of collectors
        }

    def __init__(self):

        check_reqs(self)

        scripts = {}
        for index, script in enumerate(self.scripts):
            scriptname = 'observerscript_' + str(index)
            scripts[scriptname] = script

        inputs = {}
        for index, inputDict in enumerate(inputs):
            inputName = 'observerinputs_' + str(index)
            inputs[inputName] = inputDict

        # Making stamps and stuff

        _model_inputs, _model_stamps, _model_scripts = \
            _scripts_and_stamps(system, initials)

        inHashID = 'pemod_' + _model_stamps['allstamp'][1]

        stamps = {}
        if uw.mpi.rank == 0:
            stamps = {
                'inputs': hashstamp(self.inputs),
                'scripts': hashstamp(
                    [open(script) for scriptname, script \
                        in sorted(scripts.items())]
                    ),
                'inHash': _model_stamps['allstamp'][0]
                }
            stamps['allstamp'] = hashstamp(stamps)
            for stampKey, stampVal in stamps.items():
                stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
        stamps = uw.mpi.comm.bcast(stamps, root = 0)
        stamps['inStamps'] = _model_stamps

        scripts.update(_model_scripts)
        inputs.update(_model_inputs)

        instanceID = 'peobs_' + stamps['allstamp'][1]

        checkpointer = Checkpointer(
            stamps = stamps,
            varsOfState =
            step = self.system.step,
            modeltime = self.system.modeltime,
            figs = self.figs,
            dataCollectors = self.collectors,
            scripts = scripts,
            inputs = inputs,
            )

        self.instanceID = instanceID
        self.stamps = stamps
        self.scripts = scripts
        self.inputs = inputs

        self.step = 0
        self.modeltime = 0.

        self.inFrames = []
        self._is_child = False
        self._autoarchive = True

        'system', # must be a 'system'-like object
        'initials', # must be dict (key = str, val = IC)
        'outputPath', # must be str
        'instanceID', # must be str
        '_is_child', # must be bool
        'inFrames', # must be list of Frames objects
        '_autoarchive', # must be bool
        'checkpoint', # must take pos arg (path)
        'load_checkpoint', # must take pos arg (step)
        'stamps', # must be dict
        'step', # must be int

        super().__init__()

    def update():
        self.step = self.system.step.value
        self.modeltime = self.system.modeltime.value

    def checkpoint(self, path):
        self.checkpointer.checkpoint(path)

    def load_checkpoint(self, path):
        pass

In [ ]:
class MyObserver(Observer):

    def __init__(
            self,
            _system,
            _initials,
            _outputPath,
            ):
        self.system = _system
        self.initials = _initials
        self.outputPath = _outputPath
        self.scripts = ['test.py']
        self.inputs = {'a': 1}
        super().__init__()

In [ ]:
MyObserver(system, initials, planetengine.outdir)

In [ ]:
from planetengine.utilities import Grouper
from planetengine.utilities import hashstamp
from planetengine.wordhash import wordhash as wordhashFn
from planetengine.checkpoint import Checkpointer
from planetengine.frame import _scripts_and_stamps
from planetengine.frame import _Frame
import underworld as uw
#
def make_observer_stamps_inputs(system, initials, options):
    stamps = {}
    _model_inputs, _model_stamps, _model_scripts = \
        _scripts_and_stamps(system, initials)
    if uw.mpi.rank == 0:
        stamps.update(_model_stamps)
        stamps = {
            'options': hashstamp(options),
            'scripts': hashstamp(
                [open(script) for scriptname, script \
                    in sorted(scripts.items())]
                )
            }
        stamps['allstamp'] = hashstamp(stamps)
        for stampKey, stampVal in stamps.items():
            stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
    stamps = uw.mpi.comm.bcast(stamps, root = 0)
    return stamps

class Observer(_Frame):

    figs = []
    collectors = []

    def __init__(
            self,
            system,
            initials,
            obsInps = {},
            subObservers = [],
            outputPath = '',
            _autoarchive = True,
            _parentFrame = None,
            _is_child = False,
            _autobackup = True,
            ):

        scripts = {}
        for index, subObserver in enumerate(subObservers):
            scriptname = 'observerscript_' + str(index)
            scripts[scriptname] = script

        # Making stamps and stuff

        _model_inputs, _model_stamps, _model_scripts = \
            _scripts_and_stamps(system, initials)

        self.inHashID = 'pemod_' + _model_stamps['allstamp'][1]

        stamps = {}
        if uw.mpi.rank == 0:
            stamps = {
                'obsInps': hashstamp(obsInps),
                'scripts': hashstamp(
                    [open(script) for scriptname, script \
                        in sorted(scripts.items())]
                    )
                }
            stamps['allstamp'] = hashstamp(stamps)
            for stampKey, stampVal in stamps.items():
                stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
        stamps = uw.mpi.comm.bcast(stamps, root = 0)

        # Making the checkpointer:

        checkpointer = Checkpointer(
            stamps = stamps,
            step = system.step,
            modeltime = system.modeltime,
            figs = self.figs,
            dataCollectors = self.collectors,
            scripts = scripts,
            inputs = {'obsInps': obsInps}
            )

        self.checkpointer = checkpointer
        self.inputs = inputs
        self.scripts = scripts
        self.stamps = stamps
        self.system = system
        self.initials = initials
        self.step = system.step
        self.modeltime = system.modeltime
        self.instanceID = stamps['allstamp'][1]
        self.outputPath = outputPath

        super().__init__()

    def prompt(self):
        pass

    def checkpoint(self, path):
        self.checkpointer.checkpoint(path)

    def load_checkpoint(self, path):
        pass

In [ ]:
observer = Observer(system, initials, ['./testfile.py'])

In [ ]:
path = os.path.join(outputPath, 'obstest')
observer.checkpoint(path)

In [ ]:
system.iterate()

In [ ]:
observer.checkpoint(path)